In [ ]:
import pandas as pd
import numpy as np
from scipy import stats

import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.proportion import proportions_ztest
from statsmodels.stats.multicomp import pairwise_tukeyhsd

import matplotlib.pyplot as plt
import seaborn as sns

from sqlalchemy import create_engine, text, event
import getpass  # To get the password without showing the input

from dotenv import load_dotenv
import os

load_dotenv()

import functions as f

# Access the environment variables from the .env file
password = os.environ.get('DB_PASS')


# Note that when you use _SQLAlchemy_ and establish the connection, you do not even need to be logged in Sequel Pro or MySQL Workbench.

## Connect to DB

In [ ]:
# password = getpass.getpass()

bd = "building_permits"
connection_string = 'mysql+pymysql://root:' + password + '@localhost/'+ bd
engine = create_engine(connection_string)
engine

In [ ]:
with engine.connect() as connection:
    query = text(
        'SELECT *, '
        '       bu.use as bu, '
        '       cpu.use as cpu, '
        '       bct.type as bct_type, '
        '       st.value as status_value, '
        '       firm.standardized_firm_name as firm_name, '
        '       season.name as season_name '
        'FROM record as rd '
        'JOIN status as st '
        'ON rd.status = st.id '
        'JOIN building_construction_type as bct '
        'ON rd.building_construction_type = bct.id '
        'JOIN building_use as bu '
        'ON rd.building_use = bu.id '
        'JOIN building_use as cpu '
        'ON rd.current_property_use = cpu.id '
        'LEFT JOIN costs '
        'ON rd.record_number = costs.record_number '
        'JOIN firm '
        'ON rd.standardized_firm_name = firm.id '
        'JOIN property '
        'ON rd.property = property.id '
        'JOIN season '
        'ON rd.season = season.id '
        'JOIN total_cost_bins as tcb '
        'ON rd.total_cost_bins = tcb.id;')
    result = connection.execute(query)
    df = pd.DataFrame(result.all())

# df

In [ ]:
df["status"] = df["status_value"]
df["building_construction_type"] = df["bct_type"]
df["current_property_use"] = df["cpu"]
df["building_use"] = df["bu"]
df["season"] = df["season_name"]
df["total_cost_bins"] = df["bin_name"]
df.drop(columns=[
    "id", 
    "status_value", 
    "bct_type", 
    "cpu", 
    "bu", 
    "season_name", 
    "bin_name", 
    "standardized_firm_name", 
    "use", 
    "name", 
    "property", 
    "value", 
    "type"
    ], inplace=True)

In [ ]:
df.head(1)

## Hypotesis Testing

### Paired T-Test - Comparing building cost to total cost in one/two family dwellings

In [ ]:
print("H0: There is no difference between building cost and calculated total cost")
print("H1: There is a significant difference between building cost and calculated total cost")

residential_df = df[df['building_use'] == 'One/Two-Family']

# Just select paired data (both costs)
paired_data = residential_df[['building_cost', 'calc_total_cost']].dropna()

# Make sure we have enough data
if len(paired_data) < 5:
    print(f"Insufficient data: only {len(paired_data)} records with both cost values for residential dwellings")
else:
    # Run the test
    t_stat, p_value = stats.ttest_rel(paired_data['building_cost'], paired_data['calc_total_cost'])

    print(f"Sample size (pairs): {len(paired_data)}")
    print(f"Mean building cost: ${paired_data['building_cost'].mean():.2f}")
    print(f"Mean calculated total cost: ${paired_data['calc_total_cost'].mean():.2f}")
    print(f"Mean difference: ${(paired_data['building_cost'] - paired_data['calc_total_cost']).mean():.2f}")
    print(f"Test statistic: {t_stat:.4f}")
    print(f"P-value: {p_value:.4f}")

    if p_value < 0.05:
        print("Reject H0")
        print("There is significant evidence of a difference between building cost and calculated total cost for residential dwellings")
    else:
        print("Fail to reject H0")
        print("There is not enough evidence to suggest a difference between building cost and calculated total cost for residential dwellings")

    # Create descriptive statistics for both variables
    desc_stats = paired_data.describe()
    print("\nDescriptive Statistics:")
    print(desc_stats)

    # Calculate the confidence interval for the mean difference
    mean_diff = paired_data['building_cost'] - paired_data['calc_total_cost']
    mean = mean_diff.mean()
    std_err = stats.sem(mean_diff)
    conf_interval = stats.t.interval(0.95, len(mean_diff)-1, loc=mean, scale=std_err)

    print(f"\n95% Confidence Interval for mean difference: (${conf_interval[0]:.2f}, ${conf_interval[1]:.2f})")

    # Visualize the comparison
    plt.figure(figsize=(15, 7))

    # Create a boxplot to compare distributions
    plt.subplot(1, 2, 1)
    paired_data_long = pd.melt(paired_data, var_name='Cost Type', value_name='Cost')
    sns.boxplot(x='Cost Type', y='Cost', data=paired_data_long)
    plt.title('Cost Comparison for Residential Dwellings')
    plt.ylabel('Cost ($)')
    plt.xticks(rotation=45)

    # Create a scatter plot to show the relationship
    plt.subplot(1, 2, 2)
    plt.scatter(paired_data['building_cost'], paired_data['calc_total_cost'], alpha=0.5)
    plt.plot([0, paired_data.max().max()], [0, paired_data.max().max()], 'k--', alpha=0.5)  # Diagonal line
    plt.xlabel('Building Cost ($)')
    plt.ylabel('Calculated Total Cost ($)')
    plt.title('Building vs Total Cost')
    plt.grid(True, alpha=0.3)

    


### Chi-Square Test - Exterior Changes by Season

In [ ]:
print("Does the season affect whether projects include exterior changes?")
print()
print("H0: The proportion of projects with exterior changes is the same across all seasons")
print("H1: The proportion of projects with exterior changes differs between seasons")
print("Significance level α: 0.05")
print()

df['exterior_change'] = df['change_in_exterior'].astype(bool)

# Table counting projects with and without exterior changes for each season
season_exterior_table = pd.crosstab(df['season'], df['exterior_change'])

print("Contingency Table (Count of Projects):")
print(season_exterior_table)
print()

# Percentage of projects with exterior changes for each season
percentages = {}
for season in season_exterior_table.index:
    if True in season_exterior_table.columns:
        yes_count = season_exterior_table.loc[season, True]
        total = season_exterior_table.loc[season].sum()
        percentages[season] = (yes_count / total) * 100
    else:
        percentages[season] = 0

print("Percentage of Projects with Exterior Changes:")
for season, percentage in percentages.items():
    print(f"  {season}: {percentage:.1f}%")
print()

# Chi-square test
chi2, p_value, dof, expected = stats.chi2_contingency(season_exterior_table)

print("Chi-Square Test Results:")
print(f"Chi-square statistic: {chi2:.2f}")
print(f"p-value: {p_value:.4f}")
print(f"Degrees of freedom: {dof}")
print()

# Interpretation
print("Interpretation")
if p_value < 0.05:
    print("Reject the null hypothesis.")
    print("There is significant evidence that the proportion of projects")
    print("with exterior changes varies depending on the season.")
    
    # Seasons with highest and lowest percentages
    max_season = max(percentages, key=percentages.get)
    min_season = min(percentages, key=percentages.get)
    print(f"\nThe highest percentage of exterior changes is in {max_season} ({percentages[max_season]:.1f}%)")
    print(f"The lowest percentage of exterior changes is in {min_season} ({percentages[min_season]:.1f}%)")
else:
    print("Fail to reject the null hypothesis.")
    print("There is not enough evidence to conclude that the proportion of")
    print("projects with exterior changes differs by season.")
print()

# Visualization
plt.figure(figsize=(10, 6))

plot_data = pd.DataFrame({
    'Season': list(percentages.keys()),
    'Percentage': list(percentages.values())
})

# Sort seasons
try:
    season_order = ['Winter', 'Spring', 'Summer', 'Autumn']
    plot_data['Season'] = pd.Categorical(plot_data['Season'], categories=season_order, ordered=True)
    plot_data = plot_data.sort_values('Season')
except:
    pass

sns.barplot(x='Season', y='Percentage', data=plot_data)
plt.title('Percentage of Projects with Exterior Changes by Season')
plt.xlabel('Season')
plt.ylabel('Percentage (%)')
plt.ylim(0, max(max(percentages.values()) * 1.1, 0.1))  # Set y-axis to show a bit above the max percentage
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()


### ANOVA Test - Total Cost by Number of Units

In [ ]:
# Filter to only include 1-10 units for clearer visualization
units_filter = df[(df['number_of_units'] > 0) & (df['number_of_units'] <= 10)]

print("Does the number of units affect the total cost of building permits?")

print("H0: The mean total cost is the same across all unit counts")
print("H1: At least one unit count has a different mean total cost")
print("Significance level α: 0.05")
print()

# Get count of samples
unit_counts = units_filter['number_of_units'].value_counts().sort_index()
print("Number of samples for each unit count:")
print(unit_counts)
print()

# Basic statistics for each group
grouped_stats = units_filter.groupby('number_of_units')['calc_total_cost'].agg(['mean', 'median'])
print("Mean and median total cost by number of units:")
print(grouped_stats)
print()

# Transform costs to make the data more normally distributed
units_filter['log_cost'] = np.log10(units_filter['calc_total_cost'] + 1)  # Add 1 to handle zeros
# Note: log10 is used to make values more interpretable (orders of magnitude)

# Prepare the one-way ANOVA test
groups = []
group_names = []

# Only include groups with at least 5 samples
for unit_count in sorted(units_filter['number_of_units'].unique()):
    group_data = units_filter[units_filter['number_of_units'] == unit_count]['log_cost']
    if len(group_data) >= 5:  # Only include groups with sufficient samples
        groups.append(group_data)
        group_names.append(str(unit_count))

# Run the ANOVA test
f_stat, p_value = stats.f_oneway(*groups)

print("ANOVA Results:")
print(f"F-statistic: {f_stat:.4f}")
print(f"P-value: {p_value:.4f}")
print()

# Evaluate Hypothesis
if p_value < 0.05:
    print("Reject the null hypothesis (p < 0.05)")
    print("There is significant evidence that the mean total cost differs")
    print("            based on the number of units in the building.")
else:
    print("Fail to reject the null hypothesis (p >= 0.05)")
    print("There is not enough evidence to suggest that the number of units")
    print("            affects the total cost.")
print()

# Visualizations
plt.figure(figsize=(12, 6))

# Boxplot to visualize the distribution of costs by unit count
plt.subplot(1, 2, 1)
sns.boxplot(x='number_of_units', y='calc_total_cost', data=units_filter)
plt.title('Total Cost by Number of Units')
plt.xlabel('Number of Units')
plt.ylabel('Total Cost ($)')
plt.yscale('log')  # Use log scale for better visualization of skewed data
plt.grid(True, alpha=0.3)

# Bar chart to show mean costs
plt.subplot(1, 2, 2)
mean_costs = units_filter.groupby('number_of_units')['calc_total_cost'].mean()
mean_costs.plot(kind='bar', color='skyblue')
plt.title('Mean Total Cost by Number of Units')
plt.xlabel('Number of Units')
plt.ylabel('Mean Total Cost ($)')
plt.grid(True, axis='y', alpha=0.3)
plt.tight_layout()
plt.show()
